# Implementación de un sistema de búsqueda genética basado en LangChain y Elasticsearch

## Paso 1: Configuración del Entorno
Instalamos las dependencias necesarias:

In [14]:
pip install beautifulsoup4 eland elasticsearch huggingface-hub langchain tqdm torch requests sentence_transformers googletrans==4.0.0-rc1

Note: you may need to restart the kernel to use updated packages.


## Paso 2: Traducción de la Pregunta
Utilizaremos la librería `googletrans` para traducir la pregunta:

In [15]:
pregunta = "¿Cuál es la causa genética de la fibrosis quística?"

In [16]:
from googletrans import Translator

def translate_to_english(text):
    translator = Translator()
    translation = translator.translate(text, src='es', dest='en')
    return translation.text

question_es = pregunta
question_en = translate_to_english(question_es)
print(question_en)

What is the genetic cause of cystic fibrosis?


## Paso 3
### Búsqueda de Datos en ClinVar y Carga en Elasticsearch

Este código realiza la búsqueda de datos genéticos específicos sobre una enfermedad en la base de datos de ClinVar, extrae la información relevante y la indexa en Elasticsearch para permitir búsquedas semánticas posteriores.

#### Pasos Detallados:

1. **Configuración de Elasticsearch**:
   - **Establecer Conexión**: Configuramos la conexión a Elasticsearch utilizando las credenciales y el endpoint proporcionados.
   - **Verificación de Conexión**: Verificamos que la conexión a Elasticsearch es exitosa usando el método `ping`.
   - **Creación del Índice**: Creamos un índice llamado `genetic_information` en Elasticsearch si no existe ya.

2. **Consulta a la API de ClinVar**:
   - **Buscar Enfermedad**: Realizamos una consulta en la API de ClinVar para buscar registros relacionados con una enfermedad específica (por ejemplo, "Cystic Fibrosis").
   - **Obtener Detalles**: Usamos los identificadores de los registros obtenidos en la búsqueda inicial para hacer otra solicitud a la API y obtener detalles más específicos sobre cada registro.

3. **Procesamiento y Carga de Datos en Elasticsearch**:
   - **Extracción de Información Relevante**: Procesamos la respuesta de la API para extraer datos relevantes como el título del registro, resumen, genes y mutaciones.
   - **Preparación de Datos**: Preparamos los datos en un formato adecuado para la indexación en Elasticsearch.
   - **Carga de Datos**: Usamos la función `helpers.bulk` de la biblioteca `elasticsearch` para cargar los datos extraídos en el índice de Elasticsearch.

In [17]:
enfermedad = "Cystic Fibrosis"

In [18]:
import requests
from elasticsearch import Elasticsearch, helpers
import json

# Configuración de Elasticsearch con credenciales
endpoint = "59732aade71a4aa7bf833b5c36fbc896.us-central1.gcp.cloud.es.io"
username = "elastic"
password = "7k63hJxCPlOudd62294Qlv7j"
es_url =  f"https://{username}:{password}@{endpoint}:443"

es = Elasticsearch(es_url)

# Verificar la conexión a Elasticsearch
try:
    es.ping()
    print("Conexión exitosa a Elasticsearch")
except Exception as e:
    print(f"No se pudo establecer conexión: {e}")

# Crear índice en Elasticsearch
index_name = "genetic_information"
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

# Función para consultar datos en la API de ClinVar
def fetch_clinvar_data(disease):
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=clinvar&term={disease}&retmode=json"
    response = requests.get(url)
    if response.status_code == 200:
        result = response.json()
        ids = result["esearchresult"]["idlist"]
        details_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=clinvar&id={','.join(ids)}&retmode=json"
        details_response = requests.get(details_url)
        if details_response.status_code == 200:
            return details_response.json()
    return None

# Función para cargar datos en Elasticsearch
def load_data_to_es(data, index_name):
    actions = [
        {
            "_index": index_name,
            "_source": item
        }
        for item in data
    ]
    helpers.bulk(es, actions)

# Proceso principal para buscar una enfermedad y cargar los datos
disease = enfermedad
data = fetch_clinvar_data(disease)
if data:
    records = []
    for uid in data['result']['uids']:
        record = data['result'][uid]
        records.append({
            "disease": disease,
            "title": record.get("title"),
            "summary": record.get("summary"),
            "gene": record.get("gene"),
            "mutation": record.get("variation")
        })
    load_data_to_es(records, index_name)
    print(f"Datos de {disease} cargados exitosamente en Elasticsearch")
else:
    print(f"No se encontraron datos para {disease}")


Conexión exitosa a Elasticsearch
Datos de Cystic Fibrosis cargados exitosamente en Elasticsearch


### Código para Consultar la API de ClinVar y Obtener Información Completa sobre una Enfermedad
Para simplificar el proceso, creamos una celda que consulte directamente la API de ClinVar para obtener toda la información posible sobre una enfermedad específica y la imprima en formato JSON para su inspección.


### Ejecución del Código

1. **Consulta a la API de ClinVar**:
   - **Función `fetch_clinvar_data(disease)`**: Realiza una consulta en la API de ClinVar para obtener registros relacionados con una enfermedad específica.
     - Utiliza `esearch.fcgi` para obtener los identificadores de los registros.
     - Utiliza `esummary.fcgi` para obtener detalles específicos de cada registro utilizando los identificadores obtenidos.

2. **Impresión de Datos**:
   - Guarda el json para uso posterior e imprime toda la información de la respuesta en formato JSON para inspección.



In [30]:
import requests
import json

# Función para consultar datos en la API de ClinVar
def fetch_clinvar_data(disease):
    esearch_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=clinvar&term={disease}&retmode=json"
    esearch_response = requests.get(esearch_url)
    if esearch_response.status_code == 200:
        esearch_result = esearch_response.json()
        ids = esearch_result["esearchresult"]["idlist"]
        
        if ids:
            esummary_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=clinvar&id={','.join(ids)}&retmode=json"
            esummary_response = requests.get(esummary_url)
            if esummary_response.status_code == 200:
                return esummary_response.json()
    return None

# Proceso principal para buscar una enfermedad y mostrar los datos
disease = "Cystic Fibrosis"
data = fetch_clinvar_data(disease)
if data:
    records = []
    for uid in data['result']['uids']:
        record = data['result'][uid]
        
        # Extraer información detallada sobre genes y mutaciones
        gene_info = 'N/A'
        if "genes" in record:
            genes = record["genes"]
            if genes:
                gene_info = genes[0].get('symbol', 'N/A')

        mutation_info = 'N/A'
        if "variation_set" in record:
            variations = record["variation_set"]
            if variations:
                mutation_info = variations[0].get('variation_name', 'N/A')
        
        # Intentar obtener la descripción o proporcionar una predeterminada
        description = record.get("description", "No description available.")
        if description == "N/A":
            description = record.get("summary", "No additional information provided.")

        record_data = {
            "disease": disease,
            "gene": gene_info,
            "mutation": mutation_info,
            "description": description
        }
        
        records.append(record_data)
    
    # Guardar los datos en un archivo JSON
    with open(f"{disease.replace(' ', '_').lower()}_data.json", 'w') as json_file:
        json.dump(records, json_file, indent=2)
    
    print(f"Datos de {disease} guardados exitosamente en {disease.replace(' ', '_').lower()}_data.json")
else:
    print(f"No se encontraron datos para {disease}")

# Ejemplo de lectura del archivo JSON para su posterior uso
with open(f"{disease.replace(' ', '_').lower()}_data.json", 'r') as json_file:
    loaded_data = json.load(json_file)
    for record in loaded_data:
        print(json.dumps(record, indent=2))


Datos de Cystic Fibrosis guardados exitosamente en cystic_fibrosis_data.json
{
  "disease": "Cystic Fibrosis",
  "gene": "WDR19",
  "mutation": "NM_025132.4(WDR19):c.716+2T>C",
  "description": "No description available."
}
{
  "disease": "Cystic Fibrosis",
  "gene": "TMEM67",
  "mutation": "NM_153704.6(TMEM67):c.2878G>T (p.Ala960Ser)",
  "description": "No description available."
}
{
  "disease": "Cystic Fibrosis",
  "gene": "NPHP4",
  "mutation": "NM_015102.5(NPHP4):c.3959T>C (p.Leu1320Pro)",
  "description": "No description available."
}
{
  "disease": "Cystic Fibrosis",
  "gene": "CFTR",
  "mutation": "NM_000492.4(CFTR):c.2019_2022dup (p.Ala675fs)",
  "description": "No description available."
}
{
  "disease": "Cystic Fibrosis",
  "gene": "NPHP1",
  "mutation": "NM_001128178.3(NPHP1):c.1255_1258del (p.Ser419fs)",
  "description": "No description available."
}
{
  "disease": "Cystic Fibrosis",
  "gene": "CFTR",
  "mutation": "NC_000007.13:g.(?_117120078)_(117176728_117180153)dup",
 

### Utilización de la API de ClinVar

**NOTA: Ahora usa la info del json grabado en la celda anterior**

La API de ClinVar permite acceder a datos sobre variantes genéticas y su relación con enfermedades. Usaremos las funciones `esearch` y `esummary` de la API de Entrez para obtener estos datos.

### Explicación del Código

1. **Configuración de Elasticsearch**:
   - Configuramos la conexión a Elasticsearch utilizando las credenciales y el endpoint proporcionados.
   - Verificamos la conexión y crea un índice si no existe.

2. **Consulta a la API de ClinVar**:
   - **Función `fetch_clinvar_data(disease)`**: Realiza una consulta en la API de ClinVar para obtener registros relacionados con una enfermedad específica.
     - Utiliza `esearch.fcgi` para obtener los identificadores de los registros.
     - Utiliza `esummary.fcgi` para obtener detalles específicos de cada registro utilizando los identificadores obtenidos.

3. **Procesamiento de Datos**:
   - Extrae la información relevante de cada registro, incluyendo el título, resumen, genes y mutaciones.
   - Asegura que al menos uno de los campos relevantes no sea `None` o `N/A` antes de agregar el registro a la lista de datos para cargar.

4. **Carga de Datos en Elasticsearch**:
   - Prepara y carga los datos en Elasticsearch utilizando la función `helpers.bulk`.

5. **Búsqueda y Visualización**:
   - **Función `search_data_in_es(index_name, query)`**: Realiza una búsqueda en el índice `genetic_information` para obtener documentos relacionados con "Cystic Fibrosis".
   - Imprime los resultados de manera legible, manejando los casos en los que algunos campos puedan ser `None`.

### Referencias:
- [ClinVar API Documentation](https://www.ncbi.nlm.nih.gov/clinvar/docs/maintenance_use/#api)
- [Elasticsearch Python Client Documentation](https://elasticsearch-py.readthedocs.io/en/latest/)

In [ ]:
import requests
from elasticsearch import Elasticsearch, helpers
import json

# Configuración de Elasticsearch con credenciales
endpoint = "59732aade71a4aa7bf833b5c36fbc896.us-central1.gcp.cloud.es.io"
username = "elastic"
password = "7k63hJxCPlOudd62294Qlv7j"
es_url = f"https://{username}:{password}@{endpoint}:443"

es = Elasticsearch(es_url)

# Verificar la conexión a Elasticsearch
try:
    es.ping()
    print("Conexión exitosa a Elasticsearch")
except Exception as e:
    print(f"No se pudo establecer conexión: {e}")

# Crear índice en Elasticsearch si no existe
index_name = "genetic_information"
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

# Función para cargar datos en Elasticsearch y manejar errores
def load_data_to_es(data, index_name):
    actions = [
        {
            "_index": index_name,
            "_source": item
        }
        for item in data
    ]
    try:
        helpers.bulk(es, actions)
    except helpers.BulkIndexError as e:
        print("Bulk indexing error:", e)
        for error in e.errors:
            print(json.dumps(error, indent=2))

# Cargar datos del archivo JSON guardado
def load_data_from_json(json_file_path):
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)
        load_data_to_es(data, index_name)
        print(f"Datos cargados exitosamente en Elasticsearch desde {json_file_path}")

# Proceso principal para cargar datos desde el archivo JSON
json_file_path = "cystic_fibrosis_data.json"  # Cambia esto si el nombre del archivo es diferente
load_data_from_json(json_file_path)

# Función para buscar datos en Elasticsearch
def search_data_in_es(index_name, query):
    search_query = {
        "query": {
            "match": {
                "disease": query
            }
        }
    }
    response = es.search(index=index_name, body=search_query)
    return response['hits']['hits']

# Buscar datos de "Cystic Fibrosis"
index_name = "genetic_information"
query = "Cystic Fibrosis"
results = search_data_in_es(index_name, query)

# Imprimir resultados
for result in results:
    data = result['_source']
    disease = data.get('disease', 'N/A')
    gene = data.get('gene', 'N/A')
    mutation = data.get('mutation', 'N/A')
    description = data.get('description', 'N/A')
    
    print(f"Disease: {disease}")
    print(f"Gene: {gene}")
    print(f"Mutation: {mutation}")
    print(f"Description: {description}")
    print("\n" + "-"*50 + "\n")


## Paso 4: Búsqueda Semántica con LangChain
Configuramos LangChain para realizar la búsqueda semántica en Elasticsearch:

In [7]:
import requests
from elasticsearch import Elasticsearch, helpers
import json

# Configuración de Elasticsearch con credenciales
endpoint = "59732aade71a4aa7bf833b5c36fbc896.us-central1.gcp.cloud.es.io"
username = "elastic"
password = "7k63hJxCPlOudd62294Qlv7j"
es_url =  f"https://{username}:{password}@{endpoint}:443"

es = Elasticsearch(es_url)

# Verificar la conexión a Elasticsearch
try:
    es.ping()
    print("Conexión exitosa a Elasticsearch")
except Exception as e:
    print(f"No se pudo establecer conexión: {e}")

# Crear índice en Elasticsearch
index_name = "genetic_information"
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

# Función para consultar datos en la API de ClinVar
def fetch_clinvar_data(disease):
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=clinvar&term={disease}&retmode=json"
    response = requests.get(url)
    if response.status_code == 200:
        result = response.json()
        ids = result["esearchresult"]["idlist"]
        details_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=clinvar&id={','.join(ids)}&retmode=json"
        details_response = requests.get(details_url)
        if details_response.status_code == 200:
            return details_response.json()
    return None

# Función para cargar datos en Elasticsearch
def load_data_to_es(data, index_name):
    actions = [
        {
            "_index": index_name,
            "_source": item
        }
        for item in data
    ]
    helpers.bulk(es, actions)

# Proceso principal para buscar una enfermedad y cargar los datos
disease = "Cystic Fibrosis"
data = fetch_clinvar_data(disease)
if data:
    records = []
    for uid in data['result']['uids']:
        record = data['result'][uid]
        records.append({
            "disease": disease,
            "title": record.get("title"),
            "summary": record.get("summary"),
            "gene": record.get("gene"),
            "mutation": record.get("variation")
        })
    load_data_to_es(records, index_name)
    print(f"Datos de {disease} cargados exitosamente en Elasticsearch")
else:
    print(f"No se encontraron datos para {disease}")


Conexión exitosa a Elasticsearch
Datos de Cystic Fibrosis cargados exitosamente en Elasticsearch


## Paso 5: Generación de la Respuesta
Generamos una respuesta combinando la información obtenida con la pregunta original:

In [6]:
def generate_response(question, documents):
    relevant_info = " ".join([doc['text'] for doc in documents])
    response = f"Para la pregunta '{question}', la información genética relevante es: {relevant_info}"
    return response

response = generate_response(question_es, documents)
print(response)

NameError: name 'documents' is not defined

## Paso 6: Ejecución Completa
Un script completo integrando todos los pasos anteriores:

In [ ]:
from googletrans import Translator
from elasticsearch import Elasticsearch, helpers
from langchain.retrievers import HuggingFaceElasticSearchRetriever
import json

# Función de traducción
def translate_to_english(text):
    translator = Translator()
    translation = translator.translate(text, src='es', dest='en')
    return translation.text

# Configuración de Elasticsearch
es = Elasticsearch("http://localhost:9200")
index_name = "genetic_information"

# Configurar el retriever
retriever = HuggingFaceElasticSearchRetriever(
    index_name=index_name,
    elasticsearch_url="http://localhost:9200"
)

# Función para generar respuesta
def generate_response(question, documents):
    relevant_info = " ".join([doc['text'] for doc in documents])
    response = f"Para la pregunta '{question}', la información genética relevante es: {relevant_info}"
    return response

# Pregunta original
question_es = "¿Cuál es la causa genética de la fibrosis quística?"
question_en = translate_to_english(question_es)

# Obtener documentos relevantes
documents = retriever.get_relevant_documents(question_en)

# Generar y mostrar respuesta
response = generate_response(question_es, documents)
print(response)